# 各種統一実験

## optimizer
- Adam

## learning rate
- 0.01
- 0.0001

## aggregator
- sum
- mean

## batch size
- 32

## criterion
- Cross entoropy

In [1]:
import sys
sys.path.append("../codes")

import numpy as np
import pandas as pd
import torch
import torch.optim as optim

from functools import partial
from torch_geometric.data import DataLoader
from sklearn.model_selection import KFold
from torch.utils.data.dataset import Subset

from optuna_cv_utils import cv_train, cv_test, make_datasets
from set_data_folder import make_train_data

from model import GCN
from DGCNN import DGCNN_Model
from GIN import GIN

In [5]:
torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

device(type='cuda', index=0)

In [2]:
def CV(data_list, num_epoch, model_name, adam_lr, pool="mean"):
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    
    # 交差検証
    fold = KFold(
        n_splits=10, shuffle=True, random_state=0
    )

    valid_accs = []
    for fold_idx, (train_idx, valid_idx) in enumerate(fold.split(data_list)):
        # set model
        if model_name =="GCN":
            model = GCN(hidden_channels=64,
                        num_classes=4, num_node_feature=1, pooling=pool).to(device)
        elif model_name == "DGCNN":
            model = DGCNN_Model(num_features=1, num_classes=4).to(device)
        elif model_name == "GIN":
            model = GIN(dim_features=1, dim_target=4, 
                        config={"aggregation": pool, "dropout": .5, "hidden_units": [32, 32, 32, 32],"train_eps": True}).to(device)
            
        optimizer = optim.Adam(model.parameters(), lr=adam_lr)
        criterion = torch.nn.CrossEntropyLoss()
        
        # split data
        train_loader = DataLoader(
            Subset(data_list, train_idx),
            shuffle=True,
            batch_size=32,
        )
        valid_loader = DataLoader(
            Subset(data_list, valid_idx),
            shuffle=False,
            batch_size=32,
        )

        for epoch_idx in range(num_epoch):
            # train
            cv_train(model, train_loader, device, criterion, optimizer, model_name)
            # valid
            valid_acc = cv_test(model, valid_loader, device, model_name)

        valid_accs.append(valid_acc)

    return valid_accs

In [3]:
m_list = [2, 4, 6, 8, 10]
p_list = {"100": [0.04, 0.08, 0.11, 0.15, 0.19], "1000":[0.004, 0.008, 0.012, 0.016, 0.02]}
step_list = {"100": [200, 384, 564, 736, 900], "1000":[1996, 3984, 5964, 7936, 9900]}

df = pd.DataFrame(columns=["model", "node", "p_s", "aggre", "adam_lr", "ave_acc", "std_acc"])


cnt = 0
# poisson dataset
data_list = make_datasets()
for model_name in ["GCN", "DGCNN"]:
    for adam_lr in [0.01, 0.0001]:
        if model_name == "GCN":
            for p in ["mean", "sum"]:
                valid_accs = CV(data_list, 10, model_name, adam_lr, pool=p)

                info  = {"model": model_name, "node": "100", "p_s": f"poisson", "aggre": p,
                 "adam_lr": adam_lr, "ave_acc": np.average(valid_accs), "std_acc": np.std(valid_accs)
                }

                df = df.append(pd.Series(info, name=cnt))
                cnt += 1
                
                print(round(np.average(valid_accs),2))
        else:
            valid_accs = CV(data_list, 10, model_name, adam_lr)

            info  = {"model": model_name, "node": "100", "p_s": f"poisson", "aggre": "None",
             "adam_lr": adam_lr, "ave_acc": np.average(valid_accs), "std_acc": np.std(valid_accs)
            }

            df = df.append(pd.Series(info, name=cnt))
            cnt += 1
            print(round(np.average(valid_accs),2))

# subset dataset
for node in ["100"]:
    for m, p, step in zip(m_list, p_list[node], step_list[node]):
        # make train data folder
        p_s = [
            {"kind": "barabasi", "node": [node], "p": [str(m)]},
            {"kind": "noGrowth", "node": [node], "p": [str(step)]},
            {"kind": "noAttach", "node": [node], "p": [str(m)]},
            {"kind": "random", "node": [node], "p": [str(p)]}
        ]

        # train data folderの作成
        make_train_data(p_s, "../train_data/net").copy_data()
        # dataset の作成
        data_list = make_datasets()
        
        for model_name in ["GCN", "DGCNN"]:
            for adam_lr in [0.01, 0.0001]:
                if model_name == "GCN":
                    for p in ["mean", "sum"]:
                        valid_accs = CV(data_list, 10, model_name, adam_lr, pool=p)

                        info  = {"model": model_name, "node": node, "p_s": f"{m}_{p}_{step}", "aggre": p,
                         "adam_lr": adam_lr, "ave_acc": np.average(valid_accs), "std_acc": np.std(valid_accs)
                        }

                        df = df.append(pd.Series(info, name=cnt))
                        cnt += 1
                        print(round(np.average(valid_accs),2))
                else:
                    valid_accs = CV(data_list, 10, model_name, adam_lr)

                    info  = {"model": model_name, "node": node, "p_s": f"{m}_{p}_{step}", "aggre": "None",
                     "adam_lr": adam_lr, "ave_acc": np.average(valid_accs), "std_acc": np.std(valid_accs)
                    }

                    df = df.append(pd.Series(info, name=cnt))
                    cnt += 1
                    print(round(np.average(valid_accs),2))

100%|██████████| 5000/5000 [00:18<00:00, 267.65it/s]


0.36
0.3
0.32
0.33
0.25
0.92


100%|██████████| 5000/5000 [00:09<00:00, 510.58it/s]


0.97
0.88
0.86
0.94
0.52
0.92


100%|██████████| 5000/5000 [00:16<00:00, 302.04it/s]


...

0.41
0.26
0.72
0.9
0.25
0.98


100%|██████████| 5000/5000 [00:26<00:00, 185.64it/s]


0.25
0.25
0.75
0.76
0.25
0.98


100%|██████████| 5000/5000 [00:33<00:00, 151.12it/s]


0.36
0.25
0.47
0.54
0.25
0.99


In [5]:
df.to_csv("paper_result/re_result.csv", index=False)

## GIN

In [3]:
m_list = [2, 4, 6, 8, 10]
p_list = {"100": [0.04, 0.08, 0.11, 0.15, 0.19], "1000":[0.004, 0.008, 0.012, 0.016, 0.02]}
step_list = {"100": [200, 384, 564, 736, 900], "1000":[1996, 3984, 5964, 7936, 9900]}

df = pd.DataFrame(columns=["model", "node", "p_s", "aggre", "adam_lr", "ave_acc", "std_acc"])


cnt = 0
model_name = "GIN"
# poisson dataset
data_list = make_datasets()
for adam_lr in [0.01, 0.0001]:
    for p in ["mean", "sum"]:
        valid_accs = CV(data_list, 10, model_name, adam_lr, pool=p)

        info  = {"model": model_name, "node": "100", "p_s": f"poisson", "aggre": p,
         "adam_lr": adam_lr, "ave_acc": np.average(valid_accs), "std_acc": np.std(valid_accs)
        }

        df = df.append(pd.Series(info, name=cnt))
        cnt += 1

        print(round(np.average(valid_accs),2))
    

# subset dataset
node = "100"
for m, p, step in zip(m_list, p_list[node], step_list[node]):
    # make train data folder
    p_s = [
        {"kind": "barabasi", "node": [node], "p": [str(m)]},
        {"kind": "noGrowth", "node": [node], "p": [str(step)]},
        {"kind": "noAttach", "node": [node], "p": [str(m)]},
        {"kind": "random", "node": [node], "p": [str(p)]}
    ]

    # train data folderの作成
    make_train_data(p_s, "../train_data/net").copy_data()
    # dataset の作成
    data_list = make_datasets()

    for adam_lr in [0.01, 0.0001]:
        for p in ["mean", "sum"]:
            valid_accs = CV(data_list, 10, model_name, adam_lr, pool=p)

            info  = {"model": model_name, "node": node, "p_s": f"{m}_{p}_{step}", "aggre": p,
             "adam_lr": adam_lr, "ave_acc": np.average(valid_accs), "std_acc": np.std(valid_accs)
            }

            df = df.append(pd.Series(info, name=cnt))
            cnt += 1
            print(round(np.average(valid_accs),2))

100%|██████████| 5000/5000 [00:09<00:00, 509.56it/s]


0.99
0.97
0.99
0.99


100%|██████████| 5000/5000 [00:09<00:00, 507.09it/s]


0.99
0.98
0.99
0.99


100%|██████████| 5000/5000 [00:16<00:00, 309.84it/s]


...

0.93
1.0
1.0
1.0


100%|██████████| 5000/5000 [00:26<00:00, 188.61it/s]


0.99
0.99
0.99
0.99


100%|██████████| 5000/5000 [00:33<00:00, 150.25it/s]


1.0
1.0
1.0
1.0


In [5]:
df.to_csv("paper_result/re_CV_GIN.csv", index=False)

In [6]:
df = pd.DataFrame(columns=["model", "node", "p_s", "aggre", "adam_lr", "ave_acc", "std_acc"])

cnt = 0
model_name = "GIN"
# poisson dataset
data_list = make_datasets()
for adam_lr in [0.01, 0.0001]:
    for p in ["mean", "sum"]:
        valid_accs = CV(data_list, 10, model_name, adam_lr, pool=p)

        info  = {"model": model_name, "node": "100", "p_s": f"poisson", "aggre": p,
         "adam_lr": adam_lr, "ave_acc": np.average(valid_accs), "std_acc": np.std(valid_accs)
        }

        df = df.append(pd.Series(info, name=cnt))
        cnt += 1

        print(round(np.average(valid_accs),2))

100%|██████████| 5000/5000 [00:18<00:00, 269.76it/s]


0.99
0.97
0.99
0.99


# benchmark

In [6]:
from torch_geometric.datasets import TUDataset

from sklearn.model_selection import KFold

from model import GCN
from DGCNN import DGCNN_Model

In [4]:
from torch_geometric.utils import to_networkx
import networkx as nx
from tqdm import tqdm
def make_x(data):
    G = to_networkx(data)
    new_x = torch.tensor(
        [[i] for i in list(dict(nx.degree(G)).values())],
        dtype=torch.float,
    )
    return new_x

In [13]:
def CV(data_list, num_epoch, num_class, num_feature,model_name, adam_lr, pool="mean"):
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    
    # 交差検証
    fold = KFold(
        n_splits=10, shuffle=True, random_state=0
    )

    valid_accs = []
    for fold_idx, (train_idx, valid_idx) in enumerate(fold.split(data_list)):
        # set model
        if model_name =="GCN":
            model = GCN(hidden_channels=64,
                        num_classes=num_class, num_node_feature=num_feature, pooling=pool).to(device)
        elif model_name == "DGCNN":
            model = DGCNN_Model(num_features=num_feature, num_classes=num_class).to(device)
            
        optimizer = optim.Adam(model.parameters(), lr=adam_lr)
        criterion = torch.nn.CrossEntropyLoss()
        
        # split data
        train_loader = DataLoader(
            Subset(data_list, train_idx),
            shuffle=True,
            batch_size=32,
        )
        valid_loader = DataLoader(
            Subset(data_list, valid_idx),
            shuffle=False,
            batch_size=32,
        )

        for epoch_idx in range(num_epoch):
            # train
            cv_train(model, train_loader, device, criterion, optimizer, model_name)
            # valid
            valid_acc = cv_test(model, valid_loader, device, model_name)

        valid_accs.append(valid_acc)

    return valid_accs

In [11]:
dataset = TUDataset(root='data/TUDataset', name="COLLAB")

In [12]:
dataset.num_node_features

0

In [17]:
df = pd.DataFrame(columns=["model", "name", "aggre", "adam_lr", "ave_acc", "std_acc"])

cnt = 0
for dataset_name in ["MUTAG", "REDDIT-BINARY", "DD", "COLLAB"]:
    print(dataset_name)
    dataset = TUDataset(root='data/TUDataset', name=dataset_name)
    
    torch.manual_seed(12345)
    dataset = dataset.shuffle()
    
    # node attributeがない場合 degreeをnode attribute として使用
    if dataset[0].x is None:
        for d in dataset:
            d.x = make_x(d)
    
    datalist = []
    for d in tqdm(dataset):
        if d.x is None:
            d.x = make_x(d)
        datalist.append(d)
    
    if dataset.num_node_features == 0:
        feature = 1
    else:
        feature = dataset.num_node_features
    
    
    for model_name in ["GCN", "DGCNN"]:
            for adam_lr in [0.01, 0.0001]:
                if model_name == "GCN":
                    for p in ["mean", "sum"]:
                        valid_accs = CV(datalist, 10, dataset.num_classes,
                                        feature, model_name, adam_lr, pool=p)

                        info  = {"model": model_name,  "name": dataset_name, "aggre": p,
                         "adam_lr": adam_lr, "ave_acc": np.average(valid_accs), "std_acc": np.std(valid_accs)
                        }

                        df = df.append(pd.Series(info, name=cnt))
                        cnt += 1
                        print(round(np.average(valid_accs),2))
                else:
                    valid_accs = CV(datalist, 10, dataset.num_classes, 
                                    feature, model_name, adam_lr)

                    info  = {"model": model_name, "name": dataset_name, "aggre": "None",
                     "adam_lr": adam_lr, "ave_acc": np.average(valid_accs), "std_acc": np.std(valid_accs)
                    }

                    df = df.append(pd.Series(info, name=cnt))
                    cnt += 1
                    print(round(np.average(valid_accs),2))





100%|██████████| 188/188 [00:00<00:00, 20067.93it/s]

MUTAG


0.74
0.77
0.7
0.67
0.86
0.67
REDDIT-BINARY


100%|██████████| 2000/2000 [00:03<00:00, 515.26it/s]


0.7
0.64
0.55
0.57
0.55


100%|██████████| 1178/1178 [00:00<00:00, 24523.47it/s]

0.72
DD


0.68
0.74
0.59
0.6
0.61
0.62
COLLAB


100%|██████████| 5000/5000 [00:44<00:00, 112.00it/s]


0.65
0.54
0.62
0.57
0.54
0.67


In [16]:
df.to_csv("paper_result/re_benchmark_CV.csv")

,model,name,aggre,adam_lr,ave_acc,std_acc
0,GCN,MUTAG,mean,0.0100,0.744444,0.085149
1,GCN,MUTAG,sum,0.0100,0.765497,0.106818
2,GCN,MUTAG,mean,0.0001,0.696199,0.109799
3,GCN,MUTAG,sum,0.0001,0.665497,0.128590
4,DGCNN,MUTAG,None,0.0100,0.855848,0.065094
5,DGCNN,MUTAG,None,0.0001,0.665497,0.128590
